In [7]:
import numpy as np
#in case we need to repeat experiment
#np.random.seed(255)

import pandas as pd
pd.options.display.max_rows = 22

import matplotlib.pyplot as plt
plt.style.use('classic')

import seaborn as sns
sns.set()

#sklearn imports
from sklearn.cluster import KMeans
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from pandas.api.types import CategoricalDtype
from sklearn.naive_bayes import GaussianNB
import pickle # allows for model to be saved/load to file
import time

#Use print instead of display when run as python script
pyscript = True

#Classifier verborsity where supported
verbose_level=3

#sampleN = 4300000

#Multiclass classification, binary if falase
multiclass = False
over_sample = True
balanced = True

if multiclass:
    labels=[2, 1, 0]
else:
    labels=[1, 0]

#inputfile = 'CKME136X10_2018_Data_CTF.csv'
if balanced:
    if multiclass:
        inputfile_train_O = 'CKME136X10_2018_Data_CTFB_M_O_Train.csv'
        inputfile_train_U = 'CKME136X10_2018_Data_CTFB_M_U_Train.csv'
        inputfile_test = 'CKME136X10_2018_Data_CTFB_M_Test.csv'
    else:
        inputfile_train_O = 'CKME136X10_2018_Data_CTFB_B_O_Train.csv'
        inputfile_train_U = 'CKME136X10_2018_Data_CTFB_B_U_Train.csv'
        inputfile_test = 'CKME136X10_2018_Data_CTFB_B_Test.csv'

    if over_sample:
        datafile_train = inputfile_train_O
    else:
        datafile_train = inputfile_train_U
else:
    inputfile_test = 'CKME136X10_2018_Data_Cleaned_Transformed.csv'
    df_unbalanced = 'CKME136X10_2018_Data_Cleaned_Transformed.csv'

datafile_test = inputfile_test
    
cluster = False
if (cluster):    
    datafile_train = 'CKME136X10_2018_Cluster1.csv'

cluster1_outputfile = 'CKME136X10_2018_Cluster1.csv'
cluster2_outputfile = 'CKME136X10_2018_Cluster2.csv'
cluster3_outputfile = 'CKME136X10_2018_Cluster3.csv'


    
    
model_max_iter = 1000
#datestr = 'dec_08_binary_run_100_BUD'
datestr = 'dec_11_binary_run_1000_balanced_over'

#Model Store
file_lr = 'lr_' + datestr + '.model'
file_lr_l1 = 'lr_l2_' + datestr + '.model'
file_dt = 'dt_' + datestr + '.model'
file_svm = 'svm_' + datestr + '.model'
file_knn = 'knn_' + datestr + '.model'
file_mlp = 'mlp_' + datestr + '.model'
file_kmean = 'kmean_' + datestr + '.model'
file_nbayes = 'nbayes_' + datestr + '.model'

file_final_train = 'final_train_' + datestr + '.csv'
file_final_test = 'final_test_' + datestr + '.csv'

#Enable Optimization Algorithms
enable_grid_search = False
svm_c = 1000
svm_gamma = 0.001
feature_all = True
num_clusters = 2
defaultFeatures = ['P_AGE', 'V_YEAR', 'C_HOUR', 'C_YEAR', 'C_MNTH', 'C_CONF', 'C_WDAY', 'C_VEHS', 'P_USER', 'P_SEX']

enable_lr_l1 = False
predict_lr_l1 = False

# Enable Algorithms
enable_lr = True
enable_dt = True
enable_svm = True
enable_knn = False
enable_mlp = True
enable_kmean = False
enable_nbayes = True

predict_lr = True
predict_dt = True
predict_svm = True
predict_knn = False
predict_mlp = True
predict_nbayes = True

In [8]:
# this function converts the data frame to the appropriate data type
def convert_type(data):
    data = data.astype('category')
    data['C_MNTH'] = data['C_MNTH'].astype(CategoricalDtype(ordered=True))
    data['C_WDAY'] = data['C_WDAY'].astype(CategoricalDtype(ordered=True))
    data['C_HOUR'] = data['C_HOUR'].astype(CategoricalDtype(ordered=True))
    data['C_VEHS'] = data['C_VEHS'].astype(CategoricalDtype(ordered=True))
    data['P_AGE'] = data['P_AGE'].astype(CategoricalDtype(ordered=True))
    data['P_PSN'] = data['P_PSN'].astype(CategoricalDtype(ordered=True))
    data['P_ISEV'] = data['P_ISEV'].astype('int')
    return data

In [9]:
#print("Sample size: {}".format(sampleN))

if multiclass:
    print("Multi-Class Classification: Enabled")
else:
    print("Multi-Class Classification: Disabled")

if enable_grid_search:
    print("Grid Search: Enabled")
else:
    print("Grid Search: Disabled")

if feature_all:
    print("All Features: Enabled")
else:
    print("All Features: Disabled")
    
if enable_kmean:
    print("K-means: Enabled")
else:
    print("K-means: Disabled")

if enable_lr_l1:
    print("Logistic Regression: Enabled")
else:
    print("Logistic Regression: Disabled")
    
if enable_dt:
    print("Decision Tree: Enabled")
else:
    print("Decision Tree: Disabled")
    
if enable_svm:
    print("Support Vector Machines: Enabled")
else:
    print("Support Vector Machines: Disabled")

if  enable_knn:
    print("KNN: Enabled")
else:
    print("KNN: Disabled")
    
if enable_mlp:
    print("MLP: Enabled")
else:
    print("MLP: Disabled")


Multi-Class Classification: Disabled
Grid Search: Disabled
All Features: Enabled
K-means: Disabled
Logistic Regression: Disabled
Decision Tree: Enabled
Support Vector Machines: Enabled
KNN: Disabled
MLP: Enabled


In [10]:
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ))

Tue Dec 11 19:35:53 2018


In [11]:
#load data
if balanced: 
    df_test = pd.read_csv(datafile_test, engine = 'python')
    df_train = pd.read_csv(datafile_train, engine = 'python')
    df = df_train.copy()

    print(df_test.head(2))
    print(df_train.head(2))
else:
    df_unbalanced = pd.read_csv(datafile_test, engine = 'python')
    df_tmp1 = df_unbalanced.copy()


   C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  C_RALN  \
0       7       1       2       2      22       1       3       2       1   
1       6       1       4       3      31       1       1       1       1   

   C_TRAF  P_SEX  P_AGE  P_PSN  P_USER  P_ISEV  
0       7      2      2      1       1       0  
1       7      1      4      1       2       1  
   C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  C_RALN  \
0       2       1       2       1       2       4       1       1       2   
1       7       7       3       2      21       2       3       2       1   

   C_TRAF  P_SEX  P_AGE  P_PSN  P_USER  P_ISEV  
0       7      1      4      3       3       1  
1       1      2      3      1       1       0  


In [13]:
# not the best approch, needs to be rewritten.  As the data is split before balanced, we do not need this
# step for balanced dataset

if (not balanced):

    ## Split Training and Test set 70/30 split, so we don't bleed information to test set
    #Split between data and class
    ubY = df_unbalanced[df_unbalanced.columns[-1]].copy()
    if (not multiclass):
        ubY.replace(to_replace = 1, value = 0, inplace = True)
        ubY.replace(to_replace = 2, value = 1, inplace = True)
        ubY.replace(to_replace = 3, value = 1, inplace = True)
    else:
        ubY.replace(to_replace = 1, value = 0, inplace = True)
        ubY.replace(to_replace = 2, value = 1, inplace = True)
        ubY.replace(to_replace = 3, value = 2, inplace = True)

    ubX = df_unbalanced[df_unbalanced.columns[0:df_unbalanced.columns.size -1]].copy()

    ubX_train, ubX_test, ubY_train, ubY_test = model_selection.train_test_split(ubX, ubY, test_size=0.3, stratify=ubY)
    
    df_test = ubX_test
    df_test['P_ISEV'] = ubY_test
    df_train = ubX_train
    df_train['P_ISEV'] = ubY_train

In [14]:
display(df_test)
display(df_train)

,C_MNTH,C_WDAY,C_HOUR,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,P_SEX,P_AGE,P_PSN,P_USER,P_ISEV
0,7,1,2,2,22,1,3,2,1,7,2,2,1,1,0
1,6,1,4,3,31,1,1,1,1,7,1,4,1,2,1
2,7,4,4,2,21,2,1,1,1,1,2,1,2,2,0
3,12,4,4,2,21,2,2,2,2,7,1,5,1,1,1
4,6,2,2,2,35,2,3,2,1,7,2,5,1,1,1
5,7,1,3,2,21,3,2,1,1,7,1,1,1,2,0
6,1,5,3,2,21,2,1,2,1,1,1,4,1,1,1
7,4,3,2,2,36,2,1,1,1,1,2,3,1,1,0
8,6,3,5,2,31,1,2,1,3,7,2,3,1,1,0
9,2,4,2,2,21,2,1,4,2,7,1,5,1,2,1


,C_MNTH,C_WDAY,C_HOUR,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,P_SEX,P_AGE,P_PSN,P_USER,P_ISEV
0,2,1,2,1,2,4,1,1,2,7,1,4,3,3,1
1,7,7,3,2,21,2,3,2,1,1,2,3,1,1,0
2,6,6,3,3,21,2,1,1,1,1,1,5,2,2,0
3,7,3,5,2,36,3,1,1,1,7,2,3,1,1,0
4,6,2,4,2,21,2,1,1,1,1,2,4,3,2,0
5,8,4,5,2,21,2,1,1,1,1,2,4,1,1,0
6,6,5,2,2,36,2,1,1,2,1,2,4,1,1,0
7,4,1,4,2,21,1,2,1,1,7,1,3,1,1,1
8,7,1,1,2,21,1,1,1,1,8,2,4,1,1,1
9,6,3,1,2,33,2,3,2,2,3,1,3,2,2,1


In [15]:
print(df_test.isnull().sum().sum())
print(df_train.isnull().sum().sum())

0
0


In [16]:
print(df_test[df_test.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())
print(df_train[df_train.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())

0
0


In [17]:
df_test_cat = df_test.astype('category').copy()
df_train_cat = df_train.astype('category').copy()

In [18]:
total_test_Rows = df_test_cat.index.size
print("Number of Rows in test data: {}".format(total_test_Rows))

total_train_Rows = df_train_cat.index.size
print("Number of Rows in train data: {}".format(total_train_Rows))

Number of Rows in test data: 481616
Number of Rows in train data: 4986688


In [19]:
print(df_train_cat.columns)
print(df_train_cat.dtypes)

Index(['C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG', 'C_WTHR',
       'C_RSUR', 'C_RALN', 'C_TRAF', 'P_SEX', 'P_AGE', 'P_PSN', 'P_USER',
       'P_ISEV'],
      dtype='object')
C_MNTH    category
C_WDAY    category
C_HOUR    category
C_VEHS    category
C_CONF    category
C_RCFG    category
C_WTHR    category
C_RSUR    category
C_RALN    category
C_TRAF    category
P_SEX     category
P_AGE     category
P_PSN     category
P_USER    category
P_ISEV    category
dtype: object


In [20]:
#One-Hot-Encoding of categorical
#TBD


In [21]:
#print(df_test_cat[df_test_cat.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())
#print(df_train_cat[df_train_cat.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())

### type cast train data

In [22]:
# convert to the correct type
df_train_cat = convert_type(df_train_cat)
print(df_train_cat.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4986688 entries, 0 to 4986687
Data columns (total 15 columns):
C_MNTH    category
C_WDAY    category
C_HOUR    category
C_VEHS    category
C_CONF    category
C_RCFG    category
C_WTHR    category
C_RSUR    category
C_RALN    category
C_TRAF    category
P_SEX     category
P_AGE     category
P_PSN     category
P_USER    category
P_ISEV    int32
dtypes: category(14), int32(1)
memory usage: 85.6 MB
None


### type cast test data

In [23]:
# convert to the correct type
df_test_cat = convert_type(df_test_cat)
print(df_test_cat.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481616 entries, 0 to 481615
Data columns (total 15 columns):
C_MNTH    481616 non-null category
C_WDAY    481616 non-null category
C_HOUR    481616 non-null category
C_VEHS    481616 non-null category
C_CONF    481616 non-null category
C_RCFG    481616 non-null category
C_WTHR    481616 non-null category
C_RSUR    481616 non-null category
C_RALN    481616 non-null category
C_TRAF    481616 non-null category
P_SEX     481616 non-null category
P_AGE     481616 non-null category
P_PSN     481616 non-null category
P_USER    481616 non-null category
P_ISEV    481616 non-null int32
dtypes: category(14), int32(1)
memory usage: 8.3 MB
None


## Split Training and Testing for Binary class

In [24]:
#Split between data and class for training
Y_train = df_train_cat[df_train_cat.columns[-1]]
X_train = df_train_cat[df_train_cat.columns[0:df_train_cat.columns.size -1]]

Y_test = df_test_cat[df_test_cat.columns[-1]]
X_test = df_test_cat[df_test_cat.columns[0:df_test_cat.columns.size -1]]

# split data into X and y
#X = df_sample.iloc[:,0:16]
#Y = df_sample.iloc[:,-1]

In [25]:
print(Y_train.unique())
print(Y_train.groupby(Y_train).size())
print()
print(Y_test.unique())
print(Y_test.groupby(Y_test).size())

[1 0]
P_ISEV
0    2493344
1    2493344
Name: P_ISEV, dtype: int64

[0 1]
P_ISEV
0    204577
1    277039
Name: P_ISEV, dtype: int64


In [26]:
print(X_train.head(3))

  C_MNTH C_WDAY C_HOUR C_VEHS C_CONF C_RCFG C_WTHR C_RSUR C_RALN C_TRAF P_SEX  \
0      2      1      2      1      2      4      1      1      2      7     1   
1      7      7      3      2     21      2      3      2      1      1     2   
2      6      6      3      3     21      2      1      1      1      1     1   

  P_AGE P_PSN P_USER  
0     4     3      3  
1     3     1      1  
2     5     2      2  


In [27]:
dummies = False
if (dummies):
#one hot encode train and test
    X_train = pd.get_dummies(X_train)
    X_test = pd.get_dummies(X_test)
    display(X_train)
    display(X_test)
    print(X_train.shape)

## Clustering based on K-Means Clustering

In [28]:
if enable_kmean:
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    print("K-Means Clustering: Start")
    kmeans = KMeans(n_clusters=num_clusters, init='random', n_init=10, tol=1e-04, verbose= verbose_level, max_iter=model_max_iter)
    print(kmeans)
    
    print("K-Means Clustering: Build")
    ykm = kmeans.fit(X_train)
    
    if pyscript:
        print(ykm.cluster_centers_)
        print(ykm.labels_)
    else:
        display(ykm.cluster_centers_)
        display(ykm.labels_)
    
    # save model to file
    pickle.dump(ykm, open(file_kmean, "wb"))
    
    print("K-Means Clustering: End")
    t_start =  time.time()
    
    XY_train = X_train.copy()
    XY_train['P_ISEV'] = Y_train.copy()
 
    cluster1 = XY_train[loaded_model.labels_ == 0]
    print("cluster1 Shape {}".format(cluster1.shape))
    cluster1.to_csv(cluster1_outputfile, encoding='utf-8', index=False)
    
    cluster2 = XY_train[loaded_model.labels_ == 1]
    print("cluster2 Shape {}".format(cluster2.shape))
    cluster2.to_csv(cluster2_outputfile, encoding='utf-8', index=False)

    if num_clusters == 3:
        cluster3 = XY_train[loaded_model.labels_ == 2]
        print("cluster3 Shape {}".format(cluster3.shape))
        cluster3.to_csv(cluster3_outputfile, encoding='utf-8', index=False)
        
    print(time.asctime( time.localtime(t_start) ))

    

### SVM GridSearch for Optimal Parms

In [29]:
#This operation is computationaly expensive.
if enable_grid_search:
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    param_grid = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}
    grid = GridSearchCV(SVC(), param_grid, verbose=verbose_level, n_jobs = 10)
    print(grid)
    grid.fit(X_train, Y_train)
    print(grid.best_params_)
    svm_c = grid.best_params_.get('C')
    svm_gamma = grid.best_params_.get('gamma')
    print(grid.best_estimator_)
    grid_predictions = grid.predict(X_test)
    cfn_matrix_grid = confusion_matrix(Y_test, grid_predictions)
    print(cfn_matrix_grid)
    print(classification_report(Y_test,grid_predictions))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))

## Logistic Regression Model

In [30]:
if enable_lr:
    print("Logistic Regression: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    lr = LogisticRegression(C=1, random_state=0, solver='saga', multi_class='ovr', 
                            verbose=verbose_level, n_jobs=10, max_iter=model_max_iter)
    print(lr)
    print("Logistic Regression: Fit")
    lr.fit(X_train, Y_train)
    
    # save model to file
    pickle.dump(lr, open(file_lr, "wb"))
    
    
if predict_lr:
    # load model from file
    loaded_model = pickle.load(open(file_lr, "rb"))
    print("Logistic Regression: Predict")
    y_pred = lr.predict(X_test)

    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    # print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Intercept")
    print(lr.intercept_)

    # print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Coefficients")
    print(lr.coef_)
    print()
    print("Logistic Regression: Confusion Matrix")
    cnf_matrix_lg = confusion_matrix(Y_test, y_pred, labels=labels)
    print(cnf_matrix_lg)
    print()
    print("Logistic Regression: Classification Report")
    print(classification_report(Y_test, y_pred, labels=labels))
    print()
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
print("Logistic Regression: End")

Logistic Regression: Start
Tue Dec 11 19:44:49 2018
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start=False)
Logistic Regression: Fit
convergence after 15 epochs took 47 seconds
Logistic Regression: Predict


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   47.6s finished


Accuracy of logistic regression classifier on train set: 0.63
Accuracy of logistic regression classifier on test set: 0.62
Logistic Regression: Intercept
[0.08474942]
Logistic Regression: Coefficients
[[ 0.00091436 -0.01088188 -0.0621242  -0.43409806 -0.0033331  -0.04522392
   0.02882352  0.04656657  0.19180748  0.04141639 -0.55460304  0.17503814
  -0.14709461  0.74628394]]

Logistic Regression: Confusion Matrix
[[164052 112987]
 [ 69141 135436]]

Logistic Regression: Classification Report
             precision    recall  f1-score   support

          1       0.70      0.59      0.64    277039
          0       0.55      0.66      0.60    204577

avg / total       0.64      0.62      0.62    481616


Tue Dec 11 19:45:44 2018
Logistic Regression: End


### Logistic Regression with L1 Regularization

In [31]:
if (enable_lr_l1):
    # with L1 regularization
    print("Logistic Regression with L1 Regularization: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    lr = LogisticRegression(penalty='l1', C=1, solver='saga', multi_class='ovr', 
                            verbose=verbose_level, n_jobs = 10, max_iter=model_max_iter)
    print(lr)
    print("Logistic Regression with L1 Regularization: Fit")
    lr.fit(X_train, Y_train)
    
    # save model to file
    pickle.dump(lr, open(file_lr_l1, "wb"))

if (predict_lr_l1):
    # load model from file
    loaded_model = pickle.load(open(file_lr_l1, "rb"))
    print("Logistic Regression with L1 Regularization: Predict")
    y_pred = lr.predict(X_test)
    
    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    print("Logistic Regression with L1 Regularization: Confusion Matrix")
    cnf_matrix_lg_l1 = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg_l1)
    
    print(classification_report(Y_test,y_pred))
    print("Logistic Regression with L1 Regularization: Classification Report")

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
print("Logistic Regression with L1 Regularization: End")

Logistic Regression with L1 Regularization: End


### Naive Bayes

In [32]:
# Gaussian Naive Bayes Classification
if enable_nbayes:
    print("Naive Bayes: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    nbayes = GaussianNB()
    print(nbayes)
    print("Naive Bayes: Fit")
    nbayes.fit(X_train, Y_train)
    # save model to file
    pickle.dump(nbayes, open(file_nbayes, "wb"))

if predict_nbayes:
    # load model from file
    loaded_model = pickle.load(open(file_nbayes, "rb"))
    print("Naive Bayes: Predict")
    y_pred = nbayes.predict(X_test)
    print('Accuracy of Naive Bayes classifier on train set: {:.2f}'.format(nbayes.score(X_train, Y_train)))
    print('Accuracy of Naove Nayes classifier on test set: {:.2f}'.format(nbayes.score(X_test, Y_test)))
    
    cnf_matrix_dt = confusion_matrix(Y_test, y_pred, labels=labels)
    print("Naive Bayes: Confusion Matrix")
    print(cnf_matrix_dt)
    print("Naive Bayes: Classification Report")
    print(classification_report(Y_test,y_pred, labels=labels))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    print("Naive Bayes: End")

Naive Bayes: Start
Tue Dec 11 19:45:45 2018
GaussianNB(priors=None)
Naive Bayes: Fit
Naive Bayes: Predict
Accuracy of Naive Bayes classifier on train set: 0.60
Accuracy of Naove Nayes classifier on test set: 0.58
Naive Bayes: Confusion Matrix
[[131890 145149]
 [ 57711 146866]]
Naive Bayes: Classification Report
             precision    recall  f1-score   support

          1       0.70      0.48      0.57    277039
          0       0.50      0.72      0.59    204577

avg / total       0.61      0.58      0.58    481616

Tue Dec 11 19:45:55 2018
Naive Bayes: End


### Decision Tree

In [33]:
if enable_dt:
    print("Decision Tree: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    tree = DecisionTreeClassifier(criterion='entropy',max_depth=50)
    print(tree)
    print("Decision Tree: Fit")
    tree.fit(X_train, Y_train)
    # save model to file
    pickle.dump(tree, open(file_dt, "wb"))

if predict_dt:
    # load model from file
    loaded_model = pickle.load(open(file_dt, "rb"))
    print("Decision Tree: Predict")
    y_pred = tree.predict(X_test)
    print('Accuracy of Decision Tree classifier on train set: {:.2f}'.format(tree.score(X_train, Y_train)))
    print('Accuracy of Decision Tree classifier on test set: {:.2f}'.format(tree.score(X_test, Y_test)))
    
    print()
    cnf_matrix_dt = confusion_matrix(Y_test, y_pred, labels=labels)
    print("Decision Tree: Confusion Matrix")
    print(cnf_matrix_dt)
    print()
    print("Decision Tree: Classification Report")
    print()
    print(classification_report(Y_test,y_pred, labels=labels))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    print("Decision Tree: End")

Decision Tree: Start
Tue Dec 11 19:45:55 2018
DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=50,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
Decision Tree: Fit
Decision Tree: Predict
Accuracy of Decision Tree classifier on train set: 0.84
Accuracy of Decision Tree classifier on test set: 0.61

Decision Tree: Confusion Matrix
[[161038 116001]
 [ 70163 134414]]

Decision Tree: Classification Report

             precision    recall  f1-score   support

          1       0.70      0.58      0.63    277039
          0       0.54      0.66      0.59    204577

avg / total       0.63      0.61      0.62    481616

Tue Dec 11 19:47:02 2018
Decision Tree: End


## ANN - Multilayer Perceptron

In [34]:
if enable_mlp:
    print("Multilayer Preceptron: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    
    #mlpc = MLPClassifier(alpha=1)
    #mlpc = MLPClassifier(hidden_layer_sizes=(12, 12, 12), max_iter=model_max_iter, verbose=verbose_level)
    #mlpc = MLPClassifier(hidden_layer_sizes=(25, 25, 25), verbose=verbose_level, max_iter=model_max_iter)
    mlpc = MLPClassifier(hidden_layer_sizes=(12, 12, 12), verbose=verbose_level, max_iter=model_max_iter, tol = 0.0001)
    print(mlpc)
    #mlp = multilayer_perceptron(n_hidden =2, activation='logistic', algorithm='sgd', random_state=3)
    print("Multilayer Preceptron: fit")
    mlpc.fit(X_train, Y_train)
    
    # save model to file
    pickle.dump(mlpc, open(file_mlp, "wb"))
    
if predict_mlp:
    
    # load model from file
    loaded_model = pickle.load(open(file_mlp, "rb"))
    print("Multilayer Preceptron: Predict")
    y_pred = mlpc.predict(X_test)

    print('Accuracy of Multilayer Perceptron classifier on train set: {:.2f}'.format(mlpc.score(X_train, Y_train)))
    print('Accuracy of Multilayer Perceptron classifier on test set: {:.2f}'.format(mlpc.score(X_test, Y_test)))
    print()
    print("Multilayer Preceptron: Confusion Matrix")
    cnf_matrix_mlp = confusion_matrix(Y_test, y_pred, labels=labels)
    print(cnf_matrix_mlp)
    print()
    print("Multilayer Preceptron: Classificiation Report")
    print(classification_report(Y_test,y_pred, labels=labels))
    (print)
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Multilayer Preceptron: End")

Multilayer Preceptron: Start
Tue Dec 11 19:47:02 2018
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(12, 12, 12), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=3, warm_start=False)
Multilayer Preceptron: fit
Iteration 1, loss = 0.62187705
Iteration 2, loss = 0.60733865
Iteration 3, loss = 0.60418287
Iteration 4, loss = 0.60298589
Iteration 5, loss = 0.60232672
Iteration 6, loss = 0.60176825
Iteration 7, loss = 0.60141796
Iteration 8, loss = 0.60118978
Iteration 9, loss = 0.60079787
Iteration 10, loss = 0.60032812
Iteration 11, loss = 0.59983032
Iteration 12, loss = 0.59957038
Iteration 13, loss = 0.59939765
Iteration 14, loss = 0.59927973
Iteration 15, loss = 0.59923698
Iteration 1

### SVM

In [35]:
if enable_svm:
    print("SVM: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    #svm = SVC(C=1, random_state=0, kernel='sigmoid', verbose=True)
    #svm = SVC(C=1, random_state=0, kernel='linear', verbose=True, cache_size=200)
    #svm = SVC(C=svm_c, gamma=svm_gamma, verbose = verbose_level)
    #SVN prediction is taking for ever, limiting the max_iter to 100 instead of -1 (no limit)
    #svm = SVC(C=1, gamma = 'auto', verbose = verbose_level, max_iter=model_max_iter)
    #svm = SVC(C=1000, gamma = 0.001, verbose = verbose_level, max_iter=model_max_iter)
    svm = SVC(C=1000, gamma = 0.001, kernel='poly', verbose = verbose_level, max_iter=model_max_iter)
    print(svm)
    print("SVM: Fit")
    svm.fit(X_train, Y_train)

    # save model to file
    pickle.dump(svm, open(file_svm, "wb"))
    
if predict_svm:
    # load model from file
    loaded_model = pickle.load(open(file_svm, "rb"))
    print("SVM: Predict")
    y_pred = svm.predict(X_test)
    
    print('Accuracy of SVM classifier on train set: {:.2f}'.format(svm.score(X_train, Y_train)))
    print('Accuracy of SVM classifier on test set: {:.2f}'.format(svm.score(X_test, Y_test)))
    
    print("SVM: Confusion Matrix")
    cnf_matrix_svm = confusion_matrix(Y_test, y_pred, labels=labels)
    print(cnf_matrix_svm)
    
    print("SVM: Classfication Report")
    print(classification_report(Y_test,y_pred, labels=labels))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("SVM: End")

SVM: Start
Tue Dec 11 19:54:46 2018
SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='poly',
  max_iter=1000, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=3)
SVM: Fit
[LibSVM]

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVM: Predict
Accuracy of SVM classifier on train set: 0.50
Accuracy of SVM classifier on test set: 0.57
SVM: Confusion Matrix
[[269946   7093]
 [199992   4585]]
SVM: Classfication Report
             precision    recall  f1-score   support

          1       0.57      0.97      0.72    277039
          0       0.39      0.02      0.04    204577

avg / total       0.50      0.57      0.43    481616

Tue Dec 11 20:00:37 2018
SVM: End


### K-N-N

In [36]:
if enable_knn:
    print("KNN: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski', n_jobs = -1)
    print(knn)
    print("KNN: Fit")
    knn.fit(X_train, Y_train)

    # save model to file
    pickle.dump(knn, open(file_knn, "wb"))

if predict_knn:
    # load model from file
    loaded_model = pickle.load(open(file_knn, "rb"))
    
    print("KNN: Predict")
    y_pred = knn.predict(X_test)
    print('Accuracy of KNN classifier on train set: {:.2f}'.format(knn.score(X_train, Y_train)))
    print('Accuracy of KNN classifier on test set: {:.2f}'.format(knn.score(X_test, Y_test)))
    print()
    print("KNN: Confusion Matrix")
    cnf_matrix_knn = confusion_matrix(Y_test, y_pred, labels=labels)
    print(cnf_matrix_knn)
    print()
    print("KNN: Classification Report")
    print(classification_report(Y_test,y_pred, labels=labels))
    print()
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))

    print("KNN: End")

In [37]:
t_end =  time.time()
print(time.asctime( time.localtime(t_end) ))

Tue Dec 11 20:00:37 2018
